## WhiteBox Error and Sensitivity Analysis on Wine Quality Data

Goals of tutorial include:
* [Importing wine quality dataset](#wine_quality)
* [Handling categorical features inherent within data by using dummy variables and pandas categorical datatype](#categorical)
* [Build model](#model)
* [Deploying WhiteBoxError graphics](#wbox_error)
* [Deploying WhiteBoxSensitivity graphics](#wbox_sensitivity)

In [1]:
# remove when actual package
%load_ext autoreload

%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

module_path = os.path.abspath('/projects/us_eminence/WhiteBox_Production')
if module_path not in sys.path:
    sys.path.append(module_path)

In [24]:
from whitebox import utils
from whitebox.WhiteBox import WhiteBoxError, WhiteBoxSensitivity

### Import wine quality dataset <a id=wine_quality><a>
Perform basic exploratory data analysis to better understand what types of columns are available

In [3]:
df = pd.read_csv('./datasets/winequality.csv')

In [4]:
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,quality,Type,AlcoholContent
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,5,Red,Low
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,5,Red,Low
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,6,Red,Low
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low


In [5]:
df.dtypes # it looks like most of our columns are numeric, with the exception of Type and AlcoholContent

fixed.acidity           float64
volatile.acidity        float64
citric.acid             float64
residual.sugar          float64
chlorides               float64
free.sulfur.dioxide     float64
total.sulfur.dioxide    float64
density                 float64
pH                      float64
sulphates               float64
quality                   int64
Type                     object
AlcoholContent           object
dtype: object

In [6]:
df.groupby('AlcoholContent')['fixed.acidity'].count() # most of our data resides in low/medium alcohol content

AlcoholContent
High       852
Low       2832
Medium    2813
Name: fixed.acidity, dtype: int64

In [7]:
df.groupby('Type')['fixed.acidity'].count() # and most of our data is white wine

Type
Red      1599
White    4898
Name: fixed.acidity, dtype: int64

### Handling categorical data <a id=categorical><a>

In [8]:
# we need to convert the categorical columns to the special pandas categorical datatype

# first need to identify these string columns from before
string_cols = df.select_dtypes(include = ['O'])

# convert these into categorical datatype
for cat in string_cols:
    df[cat] = pd.Categorical(df[cat])

In [9]:
df.dtypes # great - now they are categorical datatypes

fixed.acidity            float64
volatile.acidity         float64
citric.acid              float64
residual.sugar           float64
chlorides                float64
free.sulfur.dioxide      float64
total.sulfur.dioxide     float64
density                  float64
pH                       float64
sulphates                float64
quality                    int64
Type                    category
AlcoholContent          category
dtype: object

### Build model <a id=model><a>

In [10]:
# we need to define what our dependent variable is
ydepend = 'quality' # we are going to treat quality as a continuous variable for the purposes of the tutorial

# subset the training data
x_train = df.loc[:, df.columns != ydepend].copy(deep = True)

# convert these into their categorical codes using utility function convert_categorical_indpendent
x_train = utils.convert_categorical_independent(x_train)

# pull out the dependent variable column
y_train = df.loc[:, ydepend]

In [11]:
# build the model
modelobj = RandomForestRegressor()

modelobj.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [12]:
df.head(1)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,quality,Type,AlcoholContent
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low


### Create WhiteBoxError <a id=wbox_error><a>

In [13]:
# specify featuredict as a subset of columns we want to focus on
featuredict = {'fixed.acidity': 'FIXED ACIDITY',
               'Type': 'TYPE',
               'quality': 'SUPERQUALITY',
               'AlcoholContent': 'AC',
               'sulphates': 'SULPHATES',
              'volatile.acidity': 'VOLATILE ACIDITY',
              'residual.sugar': 'RESIDUAL SUGAR',
              'free.sulfur.dioxide': 'FREE SULFUR DIOXIDE'}

# specify the groupby variables
groupbyvars = ['Type']

# instantiate wbox error
WB = WhiteBoxError(modelobj = modelobj,
                   model_df = x_train,
                   ydepend= ydepend,
                   cat_df = df,
                   groupbyvars = groupbyvars,
                   featuredict = featuredict,
                   verbose=None)

2018-02-01 15:58:46,792:[WhiteBox.py:111 -             __init__()]
                                          INFO:
Logger started....


In [14]:
# run wbox error
WB.run()

2018-02-01 15:58:49,081:[WhiteBox.py:142 -              predict()]
                                          INFO:
Creating predictions using modelobj.
                        
Modelobj class name: RandomForestRegressor
2018-02-01 15:58:49,111:[WhiteBox.py:150 -              predict()]
                                          INFO:
Assigning predictions to instance dataframe
2018-02-01 15:58:49,115:[WhiteBox.py:180 -                  run()]
                                          INFO:
Running main program. Iterating over columns and applying functions depednent on datatype
2018-02-01 15:58:49,117:[WhiteBox.py:188 -                  run()]
                                          INFO:
Currently working on column: fixed.acidity
                                
Groupby: Type

2018-02-01 15:58:49,119:[WhiteBox.py:479 -            var_check()]
                                          INFO:
Column determined as continuous datatype, transforming data for continuous column
             

2018-02-01 15:58:49,575:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:49,591:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:49,607:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_f

2018-02-01 15:58:49,899:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (22, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:49,915:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (17, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:49,931:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_f

2018-02-01 15:58:50,220:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (16, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:50,236:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (14, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:50,418:[WhiteBox.py:255 -     continuous_slice()]
                                          INFO:
Creating percentile bins for current continuous group

2018-02-01 15:58:50,710:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:50,726:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:58:50,743:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_f

...

2018-02-01 15:59:01,287:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (23, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:01,300:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (23, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:01,313:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

2018-02-01 15:59:01,736:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (52, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:01,752:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (55, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:01,768:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

2018-02-01 15:59:02,069:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (133, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:02,086:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (109, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:02,104:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values 

2018-02-01 15:59:02,407:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (56, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:02,423:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (73, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:59:02,440:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

[{'Type': 'Continuous', 'Data': [{'groupByValue': 'Red', 'errNeg': -0.3052631578947369, 'errPos': 0.2473684210526316, 'FREE SULFUR DIOXIDE': 3.0, 'predictedYSmooth': 5.752830188679246, 'groupByVarName': 'Type'}, {'groupByValue': 'Red', 'errNeg': -0.1823529411764704, 'errPos': 0.26666666666666644, 'FREE SULFUR DIOXIDE': 4.0, 'predictedYSmooth': 5.509756097560977, 'groupByVarName': 'Type'}, {'groupByValue': 'Red', 'errNeg': -0.29687499999999994, 'errPos': 0.24999999999999986, 'FREE SULFUR DIOXIDE': 5.0, 'predictedYSmooth': 5.567307692307695, 'groupByVarName': 'Type'}, {'groupByValue': 'Red', 'errNeg': -0.24799999999999997, 'errPos': 0.3361702127659574, 'FREE SULFUR DIOXIDE': 6.0, 'predictedYSmooth': 5.8733812949640285, 'groupByVarName': 'Type'}, {'groupByValue': 'Red', 'errNeg': -0.24137931034482743, 'errPos': 0.3444444444444444, 'FREE SULFUR DIOXIDE': 7.0, 'predictedYSmooth': 5.664788732394365, 'groupByVarName': 'Type'}, {'groupByValue': 'Red', 'errNeg': -0.19999999999999993, 'errPos': 

In [15]:
len(WB.outputs)

7

In [16]:
WB.save('../output/example_winequality_error.html')

2018-02-01 16:00:11,986:[WhiteBox.py:295 -                 save()]
                                          INFO:
creating html output for type: html_error
2018-02-01 16:00:11,999:[WhiteBox.py:301 -                 save()]
                                          INFO:
Writing html file out to disk


### WhiteBox Sensitivity Analysis <a id=wbox_sensitivity><a>

In [22]:
# whitebox sensitivity behaves very similarly to WhiteBox Error

# specify your dependent variable
ydepend = 'quality'


# specify groupby variables
groupbyVars = ['Type']

# we need to create dummy variables to enhance our model further
dummydf = df.copy(deep=True)

# create dummies example using all categorical columns
dummies = pd.concat([pd.get_dummies(dummydf.loc[:, col], prefix = col) for col in dummydf.select_dtypes(include = ['category']).columns], axis = 1)
finaldf = pd.concat([dummydf.select_dtypes(include = [np.number]), dummies], axis = 1)



# create train dataset for fitting model
xtrain = finaldf.loc[:, finaldf.columns != ydepend].copy(deep = True)
# create dependent variable dataset
ytrain = finaldf.loc[:, ydepend]

# fit the model
modelobj.fit(xtrain, ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [ ]:

# specify featuredict as a subset of columns we want to focus on
# specify featuredict as a subset of columns we want to focus on
featuredict = {'fixed.acidity': 'FIXED ACIDITY',
               'Type': 'TYPE',
               'quality': 'SUPERQUALITY',
               'AlcoholContent': 'AC',
               'sulphates': 'SULPHATES',
              'volatile.acidity': 'VOLATILE ACIDITY',
              'residual.sugar': 'RESIDUAL SUGAR',
              'free.sulfur.dioxide': 'FREE SULFUR DIOXIDE'}

# instantiate whitebox sensitivity
WB = WhiteBoxSensitivity(modelobj = modelobj,
                   model_df = finaldf,
                   ydepend= ydepend,
                   cat_df = df,
                   groupbyvars = groupbyvars,
                   featuredict = featuredict,
                    verbose=None)
# run
WB.run()

2018-02-01 16:08:48,791:[WhiteBox.py:142 -              predict()]
                                          INFO:
Creating predictions using modelobj.
                        
Modelobj class name: RandomForestRegressor
2018-02-01 16:08:48,819:[WhiteBox.py:150 -              predict()]
                                          INFO:
Assigning predictions to instance dataframe
2018-02-01 16:08:48,823:[WhiteBox.py:180 -                  run()]
                                          INFO:
Running main program. Iterating over columns and applying functions depednent on datatype
2018-02-01 16:08:48,825:[WhiteBox.py:188 -                  run()]
                                          INFO:
Currently working on column: fixed.acidity
                                
Groupby: Type

2018-02-01 16:08:48,828:[WhiteBox.py:697 -            var_check()]
                                          INFO:
Column determined as continuous datatype, transforming data for continuous column
             

     fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
34             5.2     0.0               5.0  Red   0.1           0
45             4.6     0.5               4.5  Red   0.4           0
94             5.0     0.3               4.3  Red   0.0           0
95             4.7    -0.2               5.8  Red   0.0           0
142            5.2     0.0               6.0  Red   0.1           0
     fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
34             5.2     0.0               5.0  Red   0.1           0
45             4.6     0.5               4.5  Red   0.4           0
94             5.0     0.3               4.3  Red   0.0           0
95             4.7    -0.2               5.8  Red   0.0           0
142            5.2     0.0               6.0  Red   0.1           0
     fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
12             5.6    -0.2               4.8  Red   0.0           1
49             5.6     0.2               5.2  Re

2018-02-01 16:08:49,272:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity
                            
Group: Type
                            
Group shape: (57, 6)
2018-02-01 16:08:49,286:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity
                            
Group: Type
                            
Group shape: (67, 6)
2018-02-01 16:08:49,299:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity

     fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
28             7.1     0.0               5.0  Red   0.0          16
154            7.1     0.0               5.0  Red   0.1          16
155            7.1     0.0               5.0  Red  -0.1          16
156            7.1     0.0               5.0  Red   0.1          16
157            7.1     0.0               5.0  Red  -0.1          16
     fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
64             7.2    -0.2               4.8  Red   0.3          17
65             7.2    -0.2               4.8  Red   0.3          17
104            7.2    -0.2               4.8  Red   0.2          17
137            7.2     0.2               5.2  Red   0.3          17
182            7.2     0.0               5.0  Red   0.1          17
    fixed.acidity  errors  predictedYSmooth Type  diff  fixed_bins
7             7.3    -0.8               6.2  Red   0.0          20
39            7.3     0.1               5.1  Red  

2018-02-01 16:08:49,475:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity
                            
Group: Type
                            
Group shape: (27, 6)
2018-02-01 16:08:49,489:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity
                            
Group: Type
                            
Group shape: (24, 6)
2018-02-01 16:08:49,503:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: fixed.acidity

...

      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1632             0.120     0.0               6.0  White   0.0           0
1700             0.120     0.3               5.3  White   0.3           0
1705             0.120     0.3               5.3  White   0.3           0
1859             0.115     0.3               6.3  White   0.0           0
1919             0.100    -0.7               6.3  White  -0.4           0
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1632             0.120     0.0               6.0  White   0.0           0
1700             0.120     0.3               5.3  White   0.3           0
1705             0.120     0.3               5.3  White   0.3           0
1859             0.115     0.3               6.3  White   0.0           0
1919             0.100    -0.7               6.3  White  -0.4           0
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1704             0.130     0.0        

2018-02-01 16:08:55,719:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volatile.acidity
                            
Group: Type
                            
Group shape: (223, 6)
2018-02-01 16:08:55,732:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volatile.acidity
                            
Group: Type
                            
Group shape: (266, 6)
2018-02-01 16:08:55,744:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volat

      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1599              0.27     0.0               6.0  White  -0.9          18
1606              0.27     0.0               6.0  White  -0.9          18
1609              0.27     0.7               5.7  White  -0.1          18
1623              0.27    -0.2               5.8  White  -0.6          18
1627              0.27    -0.1               5.9  White  -0.2          18
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1601              0.28    -0.1               5.9  White  -0.3          19
1604              0.28    -0.1               5.9  White  -0.3          19
1626              0.28     0.3               6.3  White   0.0          19
1634              0.28     0.3               5.3  White   0.1          19
1738              0.28     0.1               6.1  White  -0.2          19
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1767              0.29     0.2        

2018-02-01 16:08:55,928:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volatile.acidity
                            
Group: Type
                            
Group shape: (40, 6)
2018-02-01 16:08:55,943:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volatile.acidity
                            
Group: Type
                            
Group shape: (37, 6)
2018-02-01 16:08:55,956:[WhiteBox.py:622 -   transform_function()]
                                          INFO:
"Returning aggregate values for group of continuous variable in transform_function of WhiteBoxSensitivity.
                            
Column: volatil

      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1613              0.42     0.1               5.1  White   0.0          35
1768              0.42     0.4               5.4  White  -1.2          35
2269              0.42    -0.3               5.7  White  -0.2          35
2315              0.42    -0.2               5.8  White  -0.5          35
2395              0.42    -0.1               5.9  White  -0.5          35
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1683             0.430     0.1               5.1  White   0.3          36
1687             0.430     0.1               5.1  White   0.3          36
1731             0.425     0.2               5.2  White   0.0          36
1905             0.430     0.0               5.0  White   0.0          36
2058             0.430     0.3               5.3  White  -0.4          36
      volatile.acidity  errors  predictedYSmooth   Type  diff  fixed_bins
1684              0.44     0.0        

In [ ]:
# save the final outputs to disk
WB.save(fpath = './output/wine_quality_sensitivity.html')